<a href="https://colab.research.google.com/github/ethvedbitdesjan/SummerResearch/blob/main/ResNetSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('content/')

Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


In [2]:
%cd /content/content/MyDrive/SummerResearch

/content/content/MyDrive/SummerResearch


In [3]:
!pip install cuml-cu11 --extra-index-url=https://pypi.nvidia.com
!pip install cupy-cuda11x

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [4]:
import cuml
import cupy as cp

In [5]:
# Critical imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import cv2
import copy

In [6]:
import torch
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler

In [7]:
funny_set = set()
not_funny_set = set()
with open('labelled_data/funny_combined1.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        funny_set.add(line.strip())
with open('labelled_data/not_funny_combined1.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        not_funny_set.add(line.strip())

In [8]:
train_data = []
train_funny_data = []
train_not_funny_data = []
count = 0
added = set()
for file in os.listdir('Funny_Originals_Final'):
  img_array = Image.open(os.path.join('Funny_Originals_Final', file))
  if file in funny_set:
    added.add(file)
    train_funny_data.append((img_array, 1))
  if file in not_funny_set:
    added.add(file)
    train_not_funny_data.append((img_array, 0))
  count += 1
for file in os.listdir('Non-Funny_Modified_Final'):
  img_array = Image.open(os.path.join('Non-Funny_Modified_Final', file))
  if file in funny_set:
    added.add(file)
    train_funny_data.append((img_array, 1))
  if file in not_funny_set:
    added.add(file)
    train_not_funny_data.append((img_array, 0))
random.seed(123456)

In [9]:
# print(len(train_funny_data), len(train_not_funny_data))
# train_not_funny_data1 = random.sample(train_not_funny_data, k = len(train_funny_data)+10)
# train_data = train_funny_data + train_not_funny_data1
# len(train_data), len(train_funny_data), len(train_not_funny_data), len(train_not_funny_data1)
train_data = train_funny_data + train_not_funny_data

In [10]:
random.shuffle(train_data)

In [11]:
class FunnyNotFunnyDataset(Dataset):
    def __init__(self, data, root_dir=None, transform=None):
        self.data = data
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        image = self.data[index][0]
        if self.transform:
          image = self.transform(image)
        label = self.data[index][1]
        label_tensor = torch.zeros(1)
        if label == 1:
          label_tensor[0] = 1
        return {'image_data':image, 'label':label_tensor}

In [12]:
class ResNetAdded(torch.nn.Module):
  def __init__(self, resnet50=None, train_full=True):
    super(ResNetAdded, self).__init__()
    if resnet50:
      self.resnet50 = resnet50
    else:
      self.resnet50 = models.resnet50(pretrained=True)
    self.resnet50.fc = torch.nn.Identity()
    self.classifier = torch.nn.Linear(2048, 1)
    if not train_full:
      for param in self.resnet50.parameters():
        param.requires_grad = False

  def forward(self, x):
    """
    x: A batch of images.

    Returns: A tensor of predictions.
    """

    x = self.resnet50(x)
    preds = self.classifier(x)
    return preds

  def feature_extractor(self, x):
    features = self.resnet50(x)
    return features

In [13]:
idx_train_split = int(0.6 * len(train_data))
idx_test_split = int(0.8*len(train_data))
model = ResNetAdded()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
IMAGE_SIZE = 224
NUM_CLASSES = 1103
BATCH_SIZE = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(IMG_MEAN, IMG_STD)
])
test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(IMG_MEAN, IMG_STD)
])
train_dataset = FunnyNotFunnyDataset(train_data[:idx_train_split], transform = train_transform)
valid_dataset = FunnyNotFunnyDataset(train_data[idx_train_split:idx_test_split], transform = train_transform)
test_dataset = FunnyNotFunnyDataset(train_data[idx_test_split:] ,transform = test_transform)
train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size = BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=True)
model.to(device)

ResNetAdded(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [15]:
import gc
gc.collect()

21

In [16]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
lr_sch = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [17]:
num_epochs = 12
best_valid_loss = 10000

for epoch in range(num_epochs):
  print('epoch:', epoch)
  model.train()
  running_loss = 0.0
  total_loss = []
  lrs = []
  total_size = 0
  correct = 0
  for i, data in enumerate(train_dataloader):
    inputs = data['image_data'].to(device)
    label = data['label'].to(device)
    inputs = inputs.type(torch.cuda.FloatTensor)
    label = label.type(torch.cuda.FloatTensor)
    output = model.forward(inputs)
    gc.collect()
    del inputs
    loss = criterion(output, label)
    loss.mean().backward()
    optimizer.step()
    optimizer.zero_grad()
    total_loss.append(torch.sum(loss))
    gc.collect()
    del loss
    torch.cuda.empty_cache()
    predictions = torch.as_tensor((output - 0.5) >= 0, dtype=torch.int32)
    correct += (predictions == label).float().sum().item()
    gc.collect()
    #gpu_usage()
    del predictions
    del label
    del output
    #gpu_usage()
    torch.cuda.empty_cache()
    #print(predictions, "\n", targets, "\n", correct)
    total_size += BATCH_SIZE
    accuracy = correct/(total_size)
    #print(correct, total_size)
    lrs.append(optimizer.param_groups[0]['lr'])
  lr_sch.step()
  print('Mean Train loss:', torch.mean(torch.stack(total_loss)), 'Train Accuracy:', accuracy)
  model.eval()
  total_size = 0
  total_loss = []
  correct = 0
  with torch.no_grad():
      for data in valid_dataloader:
          gc.collect()
          torch.cuda.empty_cache()
          inputs = data["image_data"].to(device)
          targets = data["label"].to(device)

          inputs = inputs.type(torch.cuda.FloatTensor)
          targets = targets.type(torch.cuda.FloatTensor)
          #print(ids.shape, "ids")
          batch_size = inputs.size(0)

          output = model.forward(inputs)
          gc.collect()
          del inputs
          loss = criterion(output, targets)
          total_loss.append(torch.sum(loss))
          gc.collect()
          del loss
          torch.cuda.empty_cache()
          output = torch.sigmoid(output)

          predictions = torch.as_tensor((output - 0.5) > 0, dtype=torch.int32)
          if (predictions == targets).float().sum().item() > batch_size:
            print('error?')
          correct += (predictions == targets).float().sum().item()
          gc.collect()
          del predictions
          del targets
          del output
          torch.cuda.empty_cache()
          total_size += batch_size
          #gpu_usage()
      accuracy = correct/(total_size)
  if torch.sum(torch.stack(total_loss)) < best_valid_loss:
    best_valid_loss = torch.sum(torch.stack(total_loss))
    best_model_weights = copy.deepcopy(model.state_dict())
    path = f"best_model_full.bin"
    torch.save(model.state_dict(), path)
    print(f"Model Saved")
  print("Validation Loss over a batch: {:.4f}; Validation Loss over a single value: {:.4f} Validation Accuracy: {:.2f}%".format(torch.mean(torch.stack(total_loss)),(torch.mean(torch.stack(total_loss)))/(batch_size*8), accuracy*100))
  accuracy, correct, total_size

epoch: 0
Mean Train loss: tensor(0.7565, device='cuda:0', grad_fn=<MeanBackward0>) Train Accuracy: 0.5649509803921569
Model Saved
Validation Loss over a batch: 1.0631; Validation Loss over a single value: 0.0049 Validation Accuracy: 59.18%
epoch: 1
Mean Train loss: tensor(0.6893, device='cuda:0', grad_fn=<MeanBackward0>) Train Accuracy: 0.571078431372549
Model Saved
Validation Loss over a batch: 0.6590; Validation Loss over a single value: 0.0031 Validation Accuracy: 59.18%
epoch: 2
Mean Train loss: tensor(0.6845, device='cuda:0', grad_fn=<MeanBackward0>) Train Accuracy: 0.5716911764705882
Validation Loss over a batch: 0.6963; Validation Loss over a single value: 0.0032 Validation Accuracy: 59.18%
epoch: 3
Mean Train loss: tensor(0.6875, device='cuda:0', grad_fn=<MeanBackward0>) Train Accuracy: 0.5692401960784313
Validation Loss over a batch: 0.6896; Validation Loss over a single value: 0.0032 Validation Accuracy: 49.91%
epoch: 4
Mean Train loss: tensor(0.6949, device='cuda:0', grad_fn

In [18]:
model.load_state_dict(torch.load('best_model_full.bin'))
model.to(device)

ResNetAdded(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [19]:
model.eval()
total_size = 0
total_loss = []
correct = 0
all_preds = []
all_targets = []
with torch.no_grad():
    for data in test_dataloader:
        gc.collect()
        torch.cuda.empty_cache()
        inputs = data["image_data"].to(device)
        targets = data["label"].to(device)

        inputs = inputs.type(torch.cuda.FloatTensor)
        targets = targets.type(torch.cuda.FloatTensor)
        #print(ids.shape, "ids")
        batch_size = inputs.size(0)

        output = model.forward(inputs)
        gc.collect()
        del inputs
        loss = criterion(output, targets)
        total_loss.append(torch.sum(loss))
        gc.collect()
        del loss
        torch.cuda.empty_cache()
        output = torch.sigmoid(output)

        predictions = torch.as_tensor((output - 0.5) > 0, dtype=torch.int32)
        if (predictions == targets).float().sum().item() > batch_size:
          print('error?')
        all_preds += predictions.flatten().cpu().detach().tolist()
        all_targets += targets.flatten().cpu().detach().tolist()
        correct += (predictions == targets).float().sum().item()
        gc.collect()
        del predictions
        del targets
        del output
        torch.cuda.empty_cache()
        total_size += batch_size
        #gpu_usage()
    accuracy = correct/(total_size)
print("Test Loss over a batch: {:.4f}; Test Loss over a single value: {:.4f} Test Accuracy: {:.2f}%".format(torch.mean(torch.stack(total_loss)),(torch.mean(torch.stack(total_loss)))/(batch_size*8), accuracy*100))
accuracy, correct, total_size

Test Loss over a batch: 0.6641; Test Loss over a single value: 0.0031 Test Accuracy: 56.96%


(0.5695732838589982, 307.0, 539)

In [20]:
all_preds[0]

0

In [21]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(all_preds, all_targets), columns=['Test Not Funny', 'Test Funny'], index=['Pred Not Funny', 'Pred Funny'])

,Test Not Funny,Test Funny
Pred Not Funny,210,135
Pred Funny,97,97


In [22]:
model.eval()

ResNetAdded(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [23]:
train_dataloader = DataLoader(train_dataset, batch_size = 1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size = 1, shuffle=True)

In [24]:
funny_num = 0
for data in train_dataloader:
  targets = data["label"].to(device)
  funny_num += int(torch.sum(targets.flatten()))
funny_num

681

In [25]:
train_svm_x = []
train_svm_y = []
test_svm_x = []
test_svm_y = []
not_funny_num = 0
for data in train_dataloader:
  gc.collect()
  torch.cuda.empty_cache()
  inputs = data["image_data"].to(device)
  targets = data["label"].to(device)

  inputs = inputs.type(torch.cuda.FloatTensor)
  targets = targets.type(torch.cuda.FloatTensor)
  features = model.feature_extractor(inputs)
  features = features.flatten()
  targets = targets.flatten()
  if int(torch.sum(targets.flatten())) == 0:
    not_funny_num += 1
    if not_funny_num > funny_num:
      continue
  #print(features.shape, targets.shape)
  train_svm_x.append(cp.from_dlpack(features.detach()))
  train_svm_y.append(cp.asarray(targets))

In [26]:
import cupy as cp

In [27]:
for data in test_dataloader:
  gc.collect()
  torch.cuda.empty_cache()
  inputs = data["image_data"].to(device)
  targets = data["label"].to(device)

  inputs = inputs.type(torch.cuda.FloatTensor)
  targets = targets.type(torch.cuda.FloatTensor)
  features = model.feature_extractor(inputs)
  features = features.flatten()
  targets = targets.flatten()
  #print(features.shape, targets.shape)
  test_svm_x.append(cp.from_dlpack(features.detach()))
  test_svm_y.append(cp.asarray(targets))

In [28]:
for i in range(len(test_svm_x)):
  print(type(test_svm_x[i]), test_svm_x[i].shape)

<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class 'cupy.ndarray'> (2048,)
<class '

In [29]:
train_svm_x = cp.array(train_svm_x[:-1])
train_svm_x.shape, train_svm_x.device

((1361, 2048), <CUDA Device 0>)

In [30]:
train_svm_y, test_svm_x, test_svm_y = cp.array(train_svm_y[:-1]), cp.array(test_svm_x[:-1]), cp.array(test_svm_y[:-1])
train_svm_y.shape, test_svm_x.shape, test_svm_y.shape, train_svm_y.device, test_svm_x.device, test_svm_y.device

((1361, 1),
 (538, 2048),
 (538, 1),
 <CUDA Device 0>,
 <CUDA Device 0>,
 <CUDA Device 0>)

In [31]:
cp.save('train_svm_x.npy', train_svm_x)

In [32]:
cp.save('train_svm_y.npy', train_svm_y)
cp.save('test_svm_x.npy', test_svm_x)
cp.save('test_svm_y.npy', test_svm_y)

In [33]:
del model
del train_dataloader
del train_dataset
gc.collect()

0

In [34]:
np.sum(train_svm_y), len(train_svm_y)

(array(680., dtype=float32), 1361)

In [35]:
from cuml.svm import SVC
svm = SVC()

In [36]:
svm.fit(train_svm_x, train_svm_y)

SVC()

In [37]:
from cuml.metrics import accuracy_score
y_pred = svm.predict(test_svm_x)
accuracy_score(y_pred, test_svm_y)

0.565055787563324

In [38]:
test_svm_y.shape

(538, 1)

In [39]:
from cuml.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_pred.flatten().astype(int), test_svm_y.flatten().astype(int)).get(), columns=['Test Not Funny', 'Test Funny'], index=['Pred Not Funny', 'Pred Funny'])

,Test Not Funny,Test Funny
Pred Not Funny,124,51
Pred Funny,183,180
